In [ ]:
from cryptography import x509
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.primitives.asymmetric import rsa
from cryptography.hazmat.primitives import hashes
import datetime

## Naloga 0: Cerfitikatna agencija in certifikati končnih uporabnikov

Pri tej nalogi boste izdelali lastno certifikatno agencijo:

1. Najprej boste ustvarili samopodpisano digitalno potrdilo, ki bo predstavljalo certifikatno agencijo.
2. Zatem boste ustvarili nekaj zahtevkov za podpis certifikata (angl. certificate signing request).
3. Ter na koncu boste z zasebnim ključem certifikatne agencije podpisali zahtevke za podpis certifikata. 

Naloga je večinoma že rešena. Manjka le manjši del, ki vsakemu uporabniku na certifikatni zahtevek (in posledično certifikat) doda še njegov elektronski naslov.

To boste dosegli s spremembo implementacije metode `create_csr`: znotraj bloka `if email:` dodajte ustrezne ukaze. V pomoč poglejte v [dokumentacijo.](https://cryptography.io/en/latest/x509/) (Rešitev ni pogoj za Javanski del.)

(Izdelavo takšnih certifikatov običajno izvedemo kar preko ukaznega poziva s pomočjo programa `openssl`.)

In [ ]:
def create_ca(common_name: str):
    """Ustvari samopodpisan certifikat CA ter pripadajoč zasebni ključ."""
    # Ustvarimo zasebni ključ CA
    private_key_ca = rsa.generate_private_key(public_exponent=65537, key_size=2048)

    # Nastavimo ime CA
    x509_common_name = x509.NameAttribute(x509.NameOID.COMMON_NAME, common_name)
    subject_name_ca = x509.Name([x509_common_name])

    # Certifikat bo samopodpisan -- CA ga bo izdala sama sebi
    issuer_ca = x509.Name([x509_common_name])

    # Uporabimo pomožni objekt za izgradnjo certifikata
    builder = x509.CertificateBuilder()
    builder = builder.subject_name(subject_name_ca)
    builder = builder.issuer_name(issuer_ca)
    builder = builder.public_key(private_key_ca.public_key())
    builder = builder.serial_number(x509.random_serial_number())
    now = datetime.datetime.utcnow()
    builder = builder.not_valid_before(now)
    builder = builder.not_valid_after(now + datetime.timedelta(days=3650)) # Velja 10 let
    cert_ca = builder.sign(private_key_ca, hashes.SHA256())

    return cert_ca, private_key_ca


def create_csr(common_name: str, email: str=None):
    """Ustvari certifikatni zahtevek (angl. certificate signing requrest)
    in pripadajoč zasebni ključ."""
    # Ustvarimo zasebni ključ
    private_key_csr = rsa.generate_private_key(public_exponent=65537, key_size=2048)

    # Pripravimo pomožni objekt za izgradnjo certifikatnega zahtevka
    builder = x509.CertificateSigningRequestBuilder()

    # Naj dodamo ime kot polje COMMON NAME
    subject_name_csr = x509.Name([x509.NameAttribute(x509.NameOID.COMMON_NAME, common_name)])
    builder = builder.subject_name(subject_name_csr)

    if email:
        # TODO: Sem dodajte ukaze, ki dodajo email naslov na zatevek
        # Zapišite ga v SubjectAlternativeName kot x509.RFC822Name
        # Polje naj ima oznako nekritično (critical=False)
        pass

    # certifikatni zahtevek podpišemo
    csr = builder.sign(private_key_csr, hashes.SHA256())

    return csr, private_key_csr


def issue_certificate(cert_ca, private_key_ca, csr):
    """Vzame certifikat certifikatne agencije, njen zasebni ključ ter
    certifikatni zahtevek in slednjega podpiše z zasebnim ključem CA."""
    
    # Pripravimo pomožni objekt za izgradnjo certifikata
    builder = x509.CertificateBuilder()

    # Vsa polja iz certifikatnega zahtevka prepišemo na certifikat
    builder = builder.subject_name(csr.subject)
    builder = builder.issuer_name(cert_ca.subject)
    builder = builder.public_key(csr.public_key())
    builder = builder.serial_number(x509.random_serial_number())

    for ext in csr.extensions:
        builder = builder.add_extension(ext.value, critical=False)    

    # Certifikat bo veljaven le 1 leto
    now = datetime.datetime.utcnow()
    builder = builder.not_valid_before(now)
    builder = builder.not_valid_after(now + datetime.timedelta(days=365))

    # Vrnemo podpisan certifikat
    return builder.sign(private_key_ca, hashes.SHA256())


def save_private_key(private_key, filename):
    with open(filename, "wb") as f:
        f.write(private_key.private_bytes(
            encoding=serialization.Encoding.PEM,
            format=serialization.PrivateFormat.TraditionalOpenSSL,
            encryption_algorithm=serialization.NoEncryption()
        ))


def save_certificate(cert, filename):
    with open(filename, "wb") as f:
        f.write(cert.public_bytes(serialization.Encoding.PEM))    

Spodnja celica ustvari certifikatno agencijo ter tri uporabniške certifikate

- Certifikatna agencija z imenom `Varnost Podatkov CA`
- Uporabnik: `Ana` z email naslovom `ana@vp.si`
- Uporabnik: `Bor` z email naslovom `bor@vp.si`
- Uporabnik: `Cene` z email naslovom `cene@vp.si`

Certifikatna zahtevka Ane in Bora podpiše certifikatna agencija, certifikatni zahtevek uporabnika Cene pa podpiše uporabnik Ana.

In [ ]:
cert_ca, sk_ca = create_ca("Varnost Podatkov CA")
save_certificate(cert_ca, "cert_ca.pem")
save_private_key(sk_ca, "sk_ca.pem")

csr_ana, sk_ana = create_csr("Ana", "ana@vp.si")
save_private_key(sk_ana, "sk_ana.pem")
cert_ana = issue_certificate(cert_ca, sk_ca, csr_ana)
save_certificate(cert_ana, "cert_ana.pem")

csr_bor, sk_bor = create_csr("Bor", "bor@vp.si")
save_private_key(sk_bor, "sk_bor.pem")
cert_bor = issue_certificate(cert_ca, sk_ca, csr_bor)
save_certificate(cert_bor, "cert_bor.pem")

csr_cene, sk_cene = create_csr("Cene", "cene@vp.si")
save_private_key(sk_cene, "sk_cene.pem")
cert_cene = issue_certificate(cert_ana, sk_ana, csr_cene)
save_certificate(cert_cene, "cert_cene.pem")

Certifikati se bodo shranili v datoteke s končnico `.pem`. Te datoteke boste zatem uporabili v Javanskem programu. Spodnji test preveri, ali so email naslovi pravilno zapisani na certifikatu.

In [ ]:
assert csr_ana.extensions[0].value.get_values_for_type(x509.RFC822Name)[0] == 'ana@vp.si'
assert csr_bor.extensions[0].value.get_values_for_type(x509.RFC822Name)[0] == 'bor@vp.si'
assert csr_cene.extensions[0].value.get_values_for_type(x509.RFC822Name)[0] == 'cene@vp.si'